In [2]:
import sqlite3
import pandas as pd



# NOTA
El DataFrame df se crea a partir de la tabla basa_datos_pripal de la base de datos credit_scoring.db. El archivo no se incluye en el repositorio debido a su tamaño; la generación de la base de datos a partir del CSV original y la creación de la tabla se explica detalladamente en el notebook data-collection.ipynb

Descriptive analysis:
In this part of the project, we begin exploring the dataset created from the initial information obtained from the LendingClub dataset (Kaggle). The objective of this stage is to describe and understand the structure of the data, the variables and their types, their distributions, skewness, and the presence of missing values.

In [3]:
conn = sqlite3.connect("/workspaces/final_project_creditscoring/Data/credit_scoring.db")
df = pd.read_sql("SELECT * FROM basa_datos_pripal", conn)
conn.close()



In [4]:
conn = sqlite3.connect("/workspaces/final_project_creditscoring/Data/credit_scoring.db")
df = pd.read_sql("SELECT * FROM basa_datos_principal", conn)
conn.close()
n_rows,n_cols = df.shape

print(f'En este df existen {n_rows} filas y {n_cols} columnas')

En este df existen 192309 filas y 151 columnas


With the analysis below, we can understand that the dataset contains a large number of numerical variables, along with several categorical features represented as object types. This initial inspection highlights the need for feature selection and type handling in later stages. Now we are proceeding with a list of each type of column to identify possible issues in data types (such as date columns that are objects or numerical data that is shown as objects, etc.).

In [5]:
cols_types = df.dtypes.reset_index().rename(
    columns={'index': 'column_name', 0: 'dtype'}
)

cols_types['dtype'].value_counts()

dtype
float64    112
object      38
int64        1
Name: count, dtype: int64

In [6]:
cols_types[cols_types['dtype'] == 'object']

,column_name,dtype
1,member_id,object
5,term,object
8,grade,object
9,sub_grade,object
10,emp_title,object
11,emp_length,object
12,home_ownership,object
14,verification_status,object
15,issue_d,object
16,loan_status,object


In [7]:
print(cols_types[cols_types['dtype'] == 'float64'].to_string(index=False))

                               column_name   dtype
                                 loan_amnt float64
                               funded_amnt float64
                           funded_amnt_inv float64
                                  int_rate float64
                               installment float64
                                annual_inc float64
                                       dti float64
                               delinq_2yrs float64
                            fico_range_low float64
                           fico_range_high float64
                            inq_last_6mths float64
                    mths_since_last_delinq float64
                    mths_since_last_record float64
                                  open_acc float64
                                   pub_rec float64
                                 revol_bal float64
                                revol_util float64
                                 total_acc float64
                               

In [8]:
cols_types[cols_types['dtype'] == 'int64']

,column_name,dtype
0,id,int64


Revision of constant columns: 

The following code initially didn’t specify dropna=False, which made it show a few columns as constants. This led us to investigate what was happening and whether we were working with the right dataframe. However, this mistake was enlightening, as it helped us identify possible *data leakage variables, such as: hardship_type, deferral_term, and hardship_length.*

In [9]:
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

# Calcular número de valores únicos por columna
uniq = df.nunique(dropna=False)

# Mostrar todos los resultados
print(uniq)


id                                            192309
member_id                                          1
loan_amnt                                       1447
funded_amnt                                     1447
funded_amnt_inv                                 4951
term                                               2
int_rate                                         499
installment                                    42629
grade                                              7
sub_grade                                         35
emp_title                                      92975
emp_length                                        12
home_ownership                                     6
annual_inc                                     15656
verification_status                                3
issue_d                                          103
loan_status                                        9
pymnt_plan                                         2
url                                           

In [ ]:
uniq = df.nunique(dropna=False)
uniq[uniq == 1]

member_id      1
policy_code    1
dtype: int64

After the code revision, the only two variables with constant values are member_id and policy_code. It does not make much sense to have a unique value for member_id if we have almost 200k data entries, so we needed to check the exact values contained in this column.

In [22]:
cols = ['member_id', 'policy_code']

for col in cols:
    print(df[col].value_counts(dropna=False))

member_id
None    192309
Name: count, dtype: int64
policy_code
1.0    192309
Name: count, dtype: int64


After doing so, we discovered that it only contains None, which means the column is not relevant at all for this project and can therefore be discarded.

Regarding policy_code, a similar situation occurs. According to the data dictionary, LendingClub has only two types of policies: publicly available (1) and new products not publicly available (2). In this dataset, only publicly available products are present. Therefore, and following the same reasoning as above, policy_code is not a relevant column for the analysis.

In [23]:
df = df.drop(columns=['member_id', 'policy_code'])
df.columns

Index(['id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'emp_title',
       ...
       'hardship_payoff_balance_amount', 'hardship_last_payment_amount',
       'disbursement_method', 'debt_settlement_flag',
       'debt_settlement_flag_date', 'settlement_status', 'settlement_date',
       'settlement_amount', 'settlement_percentage', 'settlement_term'],
      dtype='object', length=149)

Revision of Duplicated Rows: No duplicated rows were identified.

In [25]:
df.duplicated().sum()

np.int64(0)

Revision of Duplicated Columns:  

In [26]:
df.T.duplicated().sum()
df.T.duplicated(keep=False)

id                                            False
loan_amnt                                     False
funded_amnt                                   False
funded_amnt_inv                               False
term                                          False
int_rate                                      False
installment                                   False
grade                                         False
sub_grade                                     False
emp_title                                     False
emp_length                                    False
home_ownership                                False
annual_inc                                    False
verification_status                           False
issue_d                                       False
loan_status                                   False
pymnt_plan                                    False
url                                           False
desc                                          False
purpose     

Two variables (deferral_term and hardship_length) were found to be exact duplicates, containing identical values across all observations. Both variables are related to post-loan hardship events (we previously identified them as potential data leakers) and will therefore be excluded from the modeling stage. 

Identify Missing Values

In [27]:
missing = df.isna().mean()*100
missing[missing>0]
print(missing)

id                                             0.000000
loan_amnt                                      0.000000
funded_amnt                                    0.000000
funded_amnt_inv                                0.000000
term                                           0.000000
int_rate                                       0.000000
installment                                    0.000000
grade                                          0.000000
sub_grade                                      0.000000
emp_title                                      6.813514
emp_length                                     5.422523
home_ownership                                 0.000000
annual_inc                                     0.000000
verification_status                            0.000000
issue_d                                        0.000000
loan_status                                    0.000000
pymnt_plan                                     0.000000
url                                            0

We identified columns with a high percentage of missing values, so we proceeded to define a missing threshold of 50%, where variables with more than 50% missing values will be considered for exclusion from the modeling stage. However, first we must evaluate them on a case-by-case basis to understand if any of those variables are conceptually important.

In [28]:
missing_threshold = 50

high_missing_cols = missing[missing >= missing_threshold]
print(high_missing_cols)

desc                                          77.978670
mths_since_last_delinq                        54.782147
mths_since_last_record                        87.175327
next_pymnt_d                                  75.705245
mths_since_last_major_derog                   80.238054
annual_inc_joint                              96.907581
dti_joint                                     96.907581
verification_status_joint                     97.014180
open_acc_6m                                   63.197250
open_act_il                                   63.197250
open_il_12m                                   63.197250
open_il_24m                                   63.197250
mths_since_rcnt_il                            64.383882
total_bal_il                                  63.197250
il_util                                       68.575574
open_rv_12m                                   63.197250
open_rv_24m                                   63.197250
max_bal_bc                                    63

In [30]:
for col in high_missing_cols.index:
    print(df[col].value_counts(dropna=False).head(10))

desc
None                                                     149960
Debt Consolidation                                            5
                                                              4
  Borrower added on 03/14/14 > Debt Consolidation<br>         3
  Borrower added on 10/25/13 > Debt consolidation<br>         3
  Borrower added on 01/14/13 > Debt consolidation<br>         3
  Borrower added on 02/19/14 > Debt consolidation<br>         3
  Borrower added on 05/13/13 > debt consolidation<br>         3
  Borrower added on 02/05/14 > Debt consolidation<br>         3
  Borrower added on 12/16/13 > Consolidate bills<br>          2
Name: count, dtype: int64
mths_since_last_delinq
NaN     105351
9.0       1648
13.0      1593
12.0      1586
14.0      1577
6.0       1556
15.0      1541
18.0      1521
8.0       1514
17.0      1513
Name: count, dtype: int64
mths_since_last_record
NaN      167646
85.0        339
100.0       339
88.0        337
89.0        335
80.0        335
78.0       

Aca me quede. Queda pendiente de todas las columnas que tienen mas de 50%, 
1. analizar por que tienen valores faltantes y si realmente debemos eliminarnas
2. Contar el total de columnas que tienen mas de 50% de valores faltantes. 
3. Con cuales nos vamos a quedar para imputar?

next steps> 
1. Valores faltantes *ongoing*
2. Otros valores faltantes *pendiente*
1. identificar filas duplicadas *hecho*
2. Sacar la columna de policy? *hecho*
3. Identificar columnas data leak *pendiente*

In [13]:
low_variance_cols = []

for col in df.columns:
    vc = df[col].value_counts(dropna=False, normalize=True)
    if vc.iloc[0] > 0.99:   # más del 99% el mismo valor
        low_variance_cols.append(col)

low_variance_cols

['member_id',
 'pymnt_plan',
 'policy_code',
 'acc_now_delinq',
 'chargeoff_within_12_mths',
 'delinq_amnt',
 'sec_app_mths_since_last_major_derog',
 'hardship_flag',
 'hardship_type',
 'hardship_reason',
 'hardship_status',
 'deferral_term',
 'hardship_amount',
 'hardship_start_date',
 'hardship_end_date',
 'payment_plan_start_date',
 'hardship_length',
 'hardship_dpd',
 'hardship_loan_status',
 'orig_projected_additional_accrued_interest',
 'hardship_payoff_balance_amount',
 'hardship_last_payment_amount']

In [14]:
desc = df.describe().T
desc.sort_values(by='max', ascending=False).head(20)

,count,mean,std,min,25%,50%,75%,max
id,192309.0,4.987646e+07,5.014201e+07,66128.0,1596224.00,3.435340e+07,9.417362e+07,1.456375e+08
annual_inc,192309.0,7.516959e+04,6.923942e+04,0.0,45000.00,6.400000e+04,9.000000e+04,8.700000e+06
tot_hi_cred_lim,147812.0,1.744076e+05,1.751981e+05,0.0,49067.75,1.120985e+05,2.542230e+05,5.850873e+06
tot_cur_bal,147812.0,1.405798e+05,1.571303e+05,0.0,28571.75,7.856900e+04,2.119948e+05,5.327039e+06
annual_inc_joint,5947.0,1.227523e+05,6.579404e+04,13464.0,82772.00,1.100000e+05,1.460000e+05,1.500000e+06
total_rev_hi_lim,147812.0,3.339641e+04,3.332634e+04,0.0,14400.00,2.470000e+04,4.150000e+04,1.417100e+06
total_bal_ex_mort,157627.0,4.885828e+04,4.721588e+04,0.0,20217.00,3.643100e+04,6.160950e+04,1.276247e+06
total_il_high_credit_limit,147812.0,4.176731e+04,4.333643e+04,0.0,14006.75,3.108400e+04,5.641425e+04,1.214546e+06
revol_bal,192309.0,1.604589e+04,2.050766e+04,0.0,5860.00,1.121500e+04,1.980900e+04,1.190046e+06
total_bal_il,70775.0,3.522341e+04,4.346104e+04,0.0,8607.50,2.289000e+04,4.601550e+04,8.017790e+05


The analysis of the maximum values reveals the presence of extreme values in some financial variables, which suggests the need to apply transformations or outlier treatment techniques in later stages.

In [15]:
desc.assign(
    mean_median_ratio = desc['mean'] / desc['50%']
).sort_values('mean_median_ratio', ascending=False).head(10)

,count,mean,std,min,25%,50%,75%,max,mean_median_ratio
delinq_2yrs,192309.0,0.269530,0.798124,0.0,0.0,0.0,0.0,26.000000,inf
pub_rec,192309.0,0.157767,0.502036,0.0,0.0,0.0,0.0,40.000000,inf
inq_last_6mths,192309.0,0.681424,0.977064,0.0,0.0,0.0,1.0,8.000000,inf
collection_recovery_fee,192309.0,22.341847,128.942794,0.0,0.0,0.0,0.0,7002.190000,inf
collections_12_mths_ex_med,192309.0,0.012158,0.132566,0.0,0.0,0.0,0.0,20.000000,inf
total_rec_late_fee,192309.0,1.415083,11.190303,0.0,0.0,0.0,0.0,1098.360001,inf
recoveries,192309.0,153.452068,774.136498,0.0,0.0,0.0,0.0,35581.880000,inf
out_prncp,192309.0,2405.221972,5863.531984,0.0,0.0,0.0,0.0,39091.640000,inf
open_il_12m,70775.0,0.670816,0.925402,0.0,0.0,0.0,1.0,20.000000,inf
acc_now_delinq,192309.0,0.003146,0.059249,0.0,0.0,0.0,0.0,3.000000,inf


Several numerical variables present highly skewed distributions, with median values equal to zero and a small proportion of non-zero observations. This pattern is expected for count-based credit history variables. However, some highly skewed variables correspond to post-loan information and will therefore be excluded from the modeling process to prevent data leakage.